In [18]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import VotingRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

Load the dataset

In [19]:
data = pd.read_csv('insurance.csv')

In [20]:
data

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


Checking missing values

In [ ]:
data.isna().sum()

Split the dataset into features and target

In [34]:
X = data.drop(columns=['charges'])
y = data['charges']

 Split the dataset into training and testing sets

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Preprocess the data using a column transformer

In [36]:
cat_features = ['sex', 'smoker', 'region']
num_features = ['age', 'bmi', 'children']
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first'), cat_features),
        ('num', StandardScaler(), num_features)
    ])

Define the base models

In [37]:
models = [
    ('dt', DecisionTreeRegressor(max_depth=5, random_state=42)),
    ('svm', SVR(kernel='rbf', gamma='scale')),
    ('nn', MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42))
]

Define the heterogeneous ensemble

In [38]:
ensemble = VotingRegressor(models)

Create the pipeline

In [39]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('ensemble', ensemble)
])

Fit the pipeline to the training data

In [40]:
pipeline.fit(X_train, y_train)

C:\Users\HP\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['sex', 'smoker', 'region']),
                                                 ('num', StandardScaler(),
                                                  ['age', 'bmi',
                                                   'children'])])),
                ('ensemble',
                 VotingRegressor(estimators=[('dt',
                                              DecisionTreeRegressor(max_depth=5,
                                                                    random_state=42)),
                                             ('svm', SVR()),
                                             ('nn',
                                              MLPRegressor(hidden_layer_sizes=(100,
                                                                               50),
             

Evaluate the pipeline on the testing data

In [41]:
y_pred = pipeline.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f'R-squared: {r2:.4f}')

R-squared: 0.7716
